<h2>Analisi esplorativa</h2>

<h4>Librerie Necessarie:</h4>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

from imblearn.over_sampling import SMOTE

<h4>Lettura del DataFrame:</h4>

In [ ]:
df = pd.read_csv("../data/star_classification.csv")

In [ ]:
df.head()

In [ ]:
df.info()

<h4>Come possiamo notare, non abbiamo valori nulli. Ottimo per poterci alleggerire la fase di pulizia dei dati</h4>

In [ ]:
df.describe()

<h3>1. PULIZIA DEI DATI</h3>

<h4>Rinominazione delle colonne: u, g, r, i, z, al fine di garantire maggiore chiarezza dei dati</h4>

In [ ]:
df.rename(columns={
    "u": "ultraviolet",
    "g": "green",
    "r": "red",
    "i": "near_infrared",
    "z": "infrared"
}, inplace=True) # con il valore andiamo ad indicare di andare a modificare il dataframe originale, senza creare una copia

<p>Spiegazione colonne DataFrame: </p>
<ul>
    <li>Alpha e delta: posizione dell'oggetto nel cielo</li>
    <li>U, g, r, i, z: spettro di luce che emana l'oggetto</li>
    <li>Class: classificazione dell'oggetto</li>
    <li>Redshift: misura quanto la luce di un oggetto è spostato verso il rosso rispetto a come viene emessa</li>
    <li>Plate: tipo di fibra usato nell'obbiettivo</li>
    <li>MJD: data in cui è stata scattata la foto, basandosi sul calendario Giuliano modificato</li>
</ul>

<h4>Eliminazione colonne superflue per l'esplorazione dei dati</h4>

In [ ]:
uselessColumns = ['run_ID', 'rerun_ID', 'cam_col', 'fiber_ID', 'field_ID']
df = df.drop(uselessColumns, axis=1)

print(f"Sono state cancellate le seguenti colonne: {uselessColumns}")

<h4>Eliminazione righe dove contengono valori sentinella. Essi sbilanciano di molto analisi future, meglio rimuoverle</h4>

In [ ]:
df = df[
    (df["ultraviolet"] > -1000) &
    (df["green"] > -1000) &
    (df["infrared"] > -1000)
]

In [ ]:
df.describe()

<h3>2. Analisi Esplorativa Dei Dati</h3>

<h4>A. Contiamo le ricorrenze della colonna target "class"</h4>

In [ ]:
sns.countplot(data=df, x="class", hue="class")
plt.title("Tipologia di oggetto cosmico",fontsize=10)
plt.show()

In [ ]:
data = df["class"].value_counts()
labels = ['GALAXY', 'STAR', 'QSO']
# define Seaborn color palette to use
colors = sns.color_palette('pastel')

# plotting data on chart
plt.pie(data, labels=labels, colors=colors, autopct='%.0f%%')
plt.show()
data

<h5>Possiamo notare come la classe prioritaria "GALAXY" sia circa il 60% dei dati complessivi, rispetto alle altre due minoritarie molto simili, "START" e "QSO", che si attestano intorno al 20%. Il DataSet non presenta un forte sbilanciamento.</h5>

<h4>B. Stampiamo la HEATMAP per verificare la correlazione delle variabili</h4>

<h5>Applichiamo una funzione encoder sulle colonne non numeriche</h5>

In [ ]:
def encode_class(value):
    if value == "GALAXY":
        return 0
    elif value == "STAR":
        return 1
    else:
        return 2
    
df_encoded = df.copy()
df_encoded["class"] = df_encoded["class"].apply(encode_class)

In [ ]:
f,ax = plt.subplots(figsize=(12,8))
sns.heatmap(df_encoded.corr(), cmap="coolwarm", annot=True, linewidths=1, fmt= '.2f',ax=ax)
plt.show()

<h4>C. Istogramma bande magnitudinali</h4>

In [ ]:
data = df[["ultraviolet", "green", "red", "near_infrared", "infrared"]].copy()



for d in data.columns:
    plt.figure(figsize=(5, 2))
    sns.histplot(x=data[d], kde=True, bins=50)
    plt.title(f"Istogramma banda magnitudinale: {d}")
    plt.show()

<h4>D. Mostriamo come, tramite scatterplot, la luminosità degli oggetti stellari cambia in relazione alla loro distanza</h4>

In [ ]:
sns.scatterplot(data=df, x="redshift", y="red", alpha=0.5)
plt.xlabel("Redshift")
plt.ylabel("Magnitudine in banda r")
plt.title("Magnitudine vs Redshift")
plt.show()


<h4>E. Classificazione oggetti stellari tramite le loro bande</h4>
<h5>Ogni stella, viene categorizzata in valori O/B, A, F, G, K, M. Essi definiscono il tipo spettrale</h5>

In [ ]:
stars = df[df["class"] == "STAR"].copy()

In [ ]:
stars["u_g"] = stars["ultraviolet"] - stars["green"]
stars["g_r"] = stars["green"] - stars["red"]
stars["r_i"] = stars["red"] - stars["near_infrared"]
stars["i_z"] = stars["near_infrared"] - stars["infrared"]

In [ ]:
def spectral_type_from_gr(g_r):
    if g_r < -0.2:
        return "O/B"
    elif g_r < 0.0:
        return "A"
    elif g_r < 0.3:
        return "F"
    elif g_r < 0.6:
        return "G"
    elif g_r < 1.0:
        return "K"
    else:
        return "M"
    
stars["spectral_type"] = stars["g_r"].apply(spectral_type_from_gr)

order = ["O/B", "A", "F", "G", "K", "M"]

stars["spectral_type"] = pd.Categorical(
    stars["spectral_type"],
    categories=order,
    ordered=True
)

In [ ]:
stars[["obj_ID", "alpha", "delta", "ultraviolet", "green", "red", "g_r", "spectral_type"]].head()

In [ ]:
sns.histplot(x=stars["spectral_type"])
plt.title("Tipi Spettrali")
plt.show()

<h5>La maggior parte di stelle all'interno del nostro DATASET hanno tipo spettrale G, simili al nostro sole!</h5>
<h5>La temperatura si aggira circa tra i 5.300 k e i 6.000 k</h5>

In [ ]:
plt.figure(figsize=(10,8))

for t in ["O/B", "A", "F", "G", "K", "M"]:
    sub = stars[stars["spectral_type"] == t]
    plt.scatter(sub["g_r"], sub["r_i"], s=5, alpha=0.5, label=t)

plt.xlabel("g − r")
plt.ylabel("r − i")
plt.title("Stellar locus by spectral type")
plt.legend(title="Spectral type")
plt.show()

<h4>F. Stima della temperatura efficace delle stelle</h4>


<h5>La formula sottostante, è una relazione empirica usata per stimare la temperatura efficace di una stella, basandosi sulla banda g_r</h5>

T<sub>eff</sub> = 10<sup>w<sub>1</sub>(g − r) + w<sub>0</sub></sup>

In [ ]:
stars = stars[stars["g_r"].between(-0.3, 1.0)]

In [ ]:
stars["Teff_K"] = 10 ** (3.877 - 0.26 * stars["g_r"])

In [ ]:
stars["Teff_K"].describe()

In [ ]:
plt.figure(figsize=(7,4))
plt.hist(stars["Teff_K"], bins=60)
plt.xlabel("Teff [K]")
plt.ylabel("Numero di stelle")
plt.title("Distribuzione della temperatura efficace")
plt.show()

In [ ]:
stars["r_i"] = stars["red"] - stars["near_infrared"]

plt.figure(figsize=(7,6))
sc = plt.scatter(
    stars["g_r"],
    stars["r_i"],
    c=stars["Teff_K"],
    s=6,
    alpha=0.5
)
plt.xlabel("g − r")
plt.ylabel("r − i")
plt.title("Locus stellare colorato per Teff")
plt.colorbar(sc, label="Teff [K]")
plt.show()

In [ ]:
gal = df[df["class"] == "GALAXY"].copy()

In [ ]:
gal["g_r"] = gal["green"] - gal["red"]

In [ ]:
gal["color_class"] = np.where(
    gal["g_r"] < 0.6,
    "Blue",
    "Red"
)

In [ ]:
sns.countplot(x="color_class", data=gal)
plt.xlabel("Galaxy class")
plt.ylabel("Count")
plt.title("Red vs Blue Galaxies")
plt.show()

In [ ]:
gal["r_i"] = gal["red"] - gal["near_infrared"]

plt.figure(figsize=(7,6))
sns.scatterplot(
    x="g_r",
    y="r_i",
    hue="color_class",
    data=gal,
    s=10,
    alpha=0.5
)
plt.xlabel("g - r")
plt.ylabel("r - i")
plt.title("Galaxy color-color diagram")
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(
    x="redshift",
    y="g_r",
    hue="color_class",
    data=gal,
    s=10,
    alpha=0.5
)
plt.xlabel("Redshift")
plt.ylabel("g − r")
plt.title("Color vs redshift")
plt.show()

In [ ]:
qso = df[df["class"] == "QSO"].copy()

In [ ]:
plt.figure(figsize=(7,4))
plt.hist(qso["redshift"], bins=50)
plt.xlabel("Redshift")
plt.ylabel("Count")
plt.title("Redshift distribution of QSO")
plt.show()

In [ ]:
qso["u_g"] = qso["ultraviolet"] - qso["green"]
qso["g_r"] = qso["green"] - qso["red"]

In [ ]:
plt.figure(figsize=(7,6))
plt.scatter(qso["u_g"], qso["g_r"], s=10, alpha=0.5)
plt.xlabel("u - g")
plt.ylabel("g - r")
plt.title("QSO color-color diagram")
plt.show()

In [ ]:
plt.figure(figsize=(10,8))

for cls, color in zip(["STAR", "GALAXY", "QSO"], ["gray", "blue", "red"]):
    sub = df[df["class"] == cls]
    plt.scatter(
        sub["ultraviolet"] - sub["green"],
        sub["green"] - sub["red"],
        s=5,
        alpha=0.4,
        label=cls
    )

plt.xlabel("u − g")
plt.ylabel("g − r")
plt.legend()
plt.title("Color–color diagram: STAR / GALAXY / QSO")
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(qso["redshift"], qso["u_g"], s=10, alpha=0.5)
plt.xlabel("Redshift")
plt.ylabel("u - g")
plt.title("QSO: color vs redshift")
plt.show()

<h2>3. Classificazione</h2>

<h4>A.1 Classificazione con modello LogisticRegression, per la previsione della colonna target "class"</h4>

<h5>A fine addestramento, il modello dovrà essere in grado di saper distinguere, tramite le colonne su cui verrà trainato, i vari oggetti stellati presenti nel nostro dataset: STAR, GALAXY e QSO</h5>

In [ ]:
df_class = df.copy()

df_class["class"]=[0 if i == "GALAXY" else 1 if i == "STAR" else 2 for i in df["class"]]

df_class = df_class.drop(["obj_ID", "alpha", "delta", "spec_obj_ID", "plate", "MJD"], axis=1)

X = df_class.drop(["class"], axis=1)
y = df_class.loc[:,'class'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

logistic_clf = LogisticRegression(random_state = 0)
logistic_clf.fit(X_train, y_train)

y_pred = logistic_clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print("Precisione rilevata: {:.2f}".format(accuracy_score(y_test, y_pred)))

ConfusionMatrixDisplay(cm).plot(cmap="Blues")
plt.show()

<h5>Come possiamo vedere dalla precisione rilevata e dalla confusion matrix, il modello è stato addestrato correttamente, avendo una precisione di 0.96</h5>

<h4>Gli unici errori che commette il modello, è quello di non riuscire ad avere una precisione ottimale sulla distinzione tra START e QSO, per via di dati molto simili tra loro e classi sbilanciate</h4>

<h4>A.2 Allenamento modello con classi bilanciate con modello SMOTE</h4>

In [ ]:
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_balanced, y_balanced, test_size = 0.25, random_state = 0)

scaler = StandardScaler()
X_train_b = scaler.fit_transform(X_train_b)
X_test_b = scaler.transform(X_test_b)

logistic_clf = LogisticRegression(random_state = 0)
logistic_clf.fit(X_train_b, y_train_b)

y_pred_b = logistic_clf.predict(X_test_b)

cm = confusion_matrix(y_test_b, y_pred_b)
print("Precisione rilevata: {:.2f}".format(accuracy_score(y_test_b, y_pred_b)))

ConfusionMatrixDisplay(cm).plot(cmap="Blues")
plt.show()

<h4>Precisione leggermente calata, solamente dello 0.01. Mentre, quello che bisogna notare è come il modello con classi bilanciate, ha abbassato il tasso di errore tra STAR e QSO</h4>

<h2>4. Regressione</h2>

<h4>Tramite il modello RandomForest, stimiamo il redshift fotometrico delle GALAXY e QSO</h4>

In [ ]:
gal_qso = df[df["class"].isin(["GALAXY", "QSO"])].copy()
gal_qso = gal_qso[(gal_qso["redshift"] >= 0) & (gal_qso["redshift"] <= 4)].copy()

gal_qso["u_g"] = gal_qso["ultraviolet"] - gal_qso["green"]
gal_qso["g_r"] = gal_qso["green"] - gal_qso["red"]
gal_qso["r_i"] = gal_qso["red"] - gal_qso["near_infrared"]
gal_qso["i_z"] = gal_qso["near_infrared"] - gal_qso["infrared"]

X = gal_qso[["u_g", "g_r", "r_i", "i_z"]]
y = gal_qso["redshift"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

model_rf = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=0)
model_rf.fit(X_train, y_train)

y_pred = model_rf.predict(X_test)

plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_pred, alpha=0.4, s=10, label="Predizioni RF")

# Aggiungiamo la linea di riferimento ideale (rossa)
lims = [0, 4]
plt.plot(lims, lims, color="red", linestyle="--", label="Perfetto (y=x)")

plt.xlabel("Redshift Vero")
plt.ylabel("Redshift Predetto (RF)")
plt.title("Stima Photo-z con Random Forest")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Errore Medio Assoluto (MAE): {mae:.4f}")
print(f"Coefficiente di Determinazione (R2): {r2:.4f}")

<h4>Risultato grafico:</h4>
<ul>
    <li>Sull'asse X sono presenti i redshift reali, dati presenti del dataset</li>
    <li>Sull'asse Y sono presenti i redshift predetti dal modello RandomForest</li>
    <li>I punti blu rappresentano gli oggetti</li>
    <li>La linea rossa rappresenta la stima perfetta. Più i punti si trovano in prossimità di essa, più si può considerare il modello preciso</li>
</ul>

<h4>Notiamo come fino al REDSHIFT 2.0, il modello è molto denso sulla retta. Successivamente inizia a disperdersi. Alla fine dell'addestramento, abbiamo ottenuto una precisione di 0.66 e un MAE di 0.19</h4>